# Tutorial 6: Time in the Quarry
## Working with Dates and Time Series

---

### 📺 Video Walkthrough

Watch the video tutorial for this notebook: [YouTube Link](#)

---

### The Price of Everything

*The Capital market moved like a living thing. Prices for Quarry specimens shifted with the winds of fashion, the appetites of senators, and the rumors that drifted up from the trappers' camps.*

*A live wharver fetched 560 units in January. By late March, after Captain Yuro's crew brought one back breathing steam, the price hit 890. Some said it would go higher still.*

*"Time changes everything," the old trader told me. "What's worthless today is treasure tomorrow. What's alive today is dead. The only constant is that the senators always want more."*

*—From the journals of a Capital price-keeper*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Convert strings to datetime objects
2. Extract date components (year, month, day, weekday)
3. Filter data by date ranges
4. Resample time series data
5. Calculate rolling statistics
6. Visualize trends over time

By the end, you will be able to answer questions like:
- How did wharver prices change over the season?
- Which days of the week see the highest trading volume?
- What's the weekly average price for each creature?

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data from GitHub
BASE_URL = "https://raw.githubusercontent.com/YOUR_USERNAME/yeller-quarry-data-science/main/data/"

prices = pd.read_csv(BASE_URL + "prices.csv")
catches = pd.read_csv(BASE_URL + "catches.csv")
creatures = pd.read_csv(BASE_URL + "creatures.csv")
incidents = pd.read_csv(BASE_URL + "incidents.csv")

print("Data loaded.")
print(f"  Price records: {len(prices)}")
print(f"  Catch records: {len(catches)}")
print(f"  Incidents: {len(incidents)}")

In [ ]:
# Look at the price data
prices.head(10)

The price records show weekly market prices in the Capital. Each row is a price quote for a creature on a specific date.

---

## Part 1: Converting to Datetime

The `date` column is currently a string. We need to convert it to a proper datetime type.

In [ ]:
# Check the current type
print(f"Current type: {prices['date'].dtype}")
print(f"Sample value: {prices['date'].iloc[0]}")

In [ ]:
# Convert to datetime
prices['date'] = pd.to_datetime(prices['date'])

print(f"New type: {prices['date'].dtype}")
print(f"Sample value: {prices['date'].iloc[0]}")

Now pandas recognizes this as a date, and we can do date-specific operations.

---

## Part 2: Extracting Date Components

The `.dt` accessor lets you extract parts of a datetime.

In [ ]:
# Extract various components
prices['year'] = prices['date'].dt.year
prices['month'] = prices['date'].dt.month
prices['day'] = prices['date'].dt.day
prices['weekday'] = prices['date'].dt.day_name()  # Monday, Tuesday, etc.
prices['week_number'] = prices['date'].dt.isocalendar().week

prices[['date', 'year', 'month', 'day', 'weekday', 'week_number']].head(10)

In [ ]:
# When are prices recorded? (Which day of the week?)
prices['weekday'].value_counts()

Prices are recorded on Fridays—the end of the Capital trading week.

---

## Part 3: Filtering by Date

With datetime columns, you can filter using comparison operators.

In [ ]:
# Prices after March 1, 1855
march_onwards = prices[prices['date'] >= '1855-03-01']
print(f"Records from March onwards: {len(march_onwards)}")
march_onwards.head()

In [ ]:
# Prices in February only
february = prices[(prices['date'] >= '1855-02-01') & (prices['date'] < '1855-03-01')]
print(f"February records: {len(february)}")

In [ ]:
# Using month component
february_v2 = prices[prices['month'] == 2]
print(f"February records (using month): {len(february_v2)}")

---

## Part 4: Time Series for a Single Creature

Let's track how wharver prices changed over the season.

In [ ]:
# Get wharver prices (creature CR009)
wharver_prices = prices[prices['creature_id'] == 'CR009'].copy()
wharver_prices = wharver_prices.sort_values('date')
print(f"Wharver price records: {len(wharver_prices)}")
wharver_prices[['date', 'price_per_unit', 'notes']]

In [ ]:
# Plot the price trend
plt.figure(figsize=(10, 5))
plt.plot(wharver_prices['date'], wharver_prices['price_per_unit'], marker='o', linewidth=2)
plt.title('Wharver Prices: January - April 1855')
plt.xlabel('Date')
plt.ylabel('Price per Unit')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

The price spike in late March—that's when Captain Yuro brought back the live specimen. Unprecedented. The notes say it burned through its cage within two days, but by then the market had moved.

---

## Part 5: Setting a DateTime Index

For time series analysis, it's often useful to make the date the DataFrame's index.

In [ ]:
# Create a time-indexed version of wharver prices
wharver_ts = wharver_prices.set_index('date')['price_per_unit']
wharver_ts

In [ ]:
# Now we can select by date directly
print("Price on March 7, 1855:")
print(wharver_ts['1855-03-07'])

In [ ]:
# Or select a range
print("Prices in March:")
print(wharver_ts['1855-03'])

---

## Part 6: Resampling Time Series

Resampling changes the frequency of your time series—for example, from weekly to monthly.

In [ ]:
# Monthly average wharver price
monthly_wharver = wharver_ts.resample('ME').mean()  # 'ME' = month end
monthly_wharver

In [ ]:
# Compare weekly vs monthly
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(wharver_ts.index, wharver_ts.values, marker='o')
axes[0].set_title('Weekly Wharver Prices')
axes[0].tick_params(axis='x', rotation=45)

axes[1].bar(monthly_wharver.index.strftime('%B'), monthly_wharver.values)
axes[1].set_title('Monthly Average Wharver Prices')

plt.tight_layout()
plt.show()

---

## Part 7: Multiple Creatures Over Time

Let's compare price trends for several creatures.

In [ ]:
# Merge creature names into prices
prices_named = prices.merge(
    creatures[['creature_id', 'common_name']], 
    on='creature_id'
)

# Get prices for a few interesting creatures
interesting = ['Wharver', 'Stone Spine Lizard', 'Leatherback Burrower', 'Yeller Bird']
subset = prices_named[prices_named['common_name'].isin(interesting)]

# Pivot to get creatures as columns
price_comparison = subset.pivot_table(
    index='date',
    columns='common_name',
    values='price_per_unit'
)

price_comparison

In [ ]:
# Plot all on one chart
plt.figure(figsize=(12, 6))
for creature in price_comparison.columns:
    plt.plot(price_comparison.index, price_comparison[creature], marker='o', label=creature)

plt.title('Creature Price Trends: January - April 1855')
plt.xlabel('Date')
plt.ylabel('Price per Unit')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

The wharver's dramatic price spike dominates the chart. Yeller birds hold steady—their value is consistent because of Senator Huilof's standing orders. Leatherback burrowers fluctuate modestly.

---

## Part 8: Rolling Statistics

Rolling calculations apply a function over a moving window.

In [ ]:
# 3-week rolling average for wharver prices
wharver_rolling = wharver_ts.rolling(window=3).mean()

plt.figure(figsize=(10, 5))
plt.plot(wharver_ts.index, wharver_ts.values, marker='o', label='Weekly Price', alpha=0.7)
plt.plot(wharver_rolling.index, wharver_rolling.values, linewidth=3, label='3-Week Rolling Average')
plt.title('Wharver Prices with Rolling Average')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

The rolling average smooths out weekly fluctuations and shows the underlying trend more clearly.

---

## Part 9: Analyzing Catches Over Time

Let's apply these techniques to the catch records.

In [ ]:
# Convert catch dates
catches['date'] = pd.to_datetime(catches['date'])
catches['total_value'] = catches['quantity'] * catches['price_per_unit']

# Daily catch count
daily_catches = catches.groupby('date').size()
daily_catches.head(20)

In [ ]:
# Weekly total revenue
weekly_revenue = catches.set_index('date')['total_value'].resample('W').sum()

plt.figure(figsize=(10, 5))
plt.bar(weekly_revenue.index, weekly_revenue.values)
plt.title('Weekly Catch Revenue')
plt.xlabel('Week')
plt.ylabel('Total Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---

## Part 10: The Incident Timeline

When do incidents happen? Is there a pattern?

In [ ]:
# Convert incident dates
incidents['date'] = pd.to_datetime(incidents['date'])
incidents['month'] = incidents['date'].dt.month

# Incidents by month
monthly_incidents = incidents.groupby('month').size()
monthly_incidents

In [ ]:
# Incidents by type over time
incident_timeline = incidents.groupby(['month', 'incident_type']).size().unstack(fill_value=0)
incident_timeline

In [ ]:
# The Redmane disaster: March 18, 1855
march_18 = incidents[incidents['date'] == '1855-03-18']
march_18[['incident_id', 'incident_type', 'severity', 'description']]

March 18, 1855. Three incidents on one day. Truck killed. The Boss missing. An equipment failure that probably saved someone's life by forcing them to stay at camp.

---

## ✏️ Exercises

### Exercise 1: Price Volatility

Which creature has the most volatile prices? Calculate the standard deviation of price_per_unit for each creature and find the highest.

In [ ]:
# Your code here



### Exercise 2: Best Week for Catches

Which week had the highest total catch value? When was it, and what was caught?

In [ ]:
# Your code here



### Exercise 3: Before and After

Compare catch activity before and after the Redmane disaster (March 18, 1855). How many catches occurred in each period? What was the total value?

In [ ]:
# Your code here



### Exercise 4: Monthly Price Changes

For the wharver, calculate the percent change in price from month to month. Use the monthly average prices.

In [ ]:
# Your code here
# Hint: Use .pct_change() on a monthly resampled series



### Exercise 5: Incident Clustering

Are there any other dates with multiple incidents? Find all dates that have more than one incident recorded.

In [ ]:
# Your code here



---

## 📋 Summary

In this tutorial, you learned:

| Concept | Code |
|---------|------|
| Convert to datetime | `pd.to_datetime(df['col'])` |
| Extract year | `df['date'].dt.year` |
| Extract month | `df['date'].dt.month` |
| Extract day name | `df['date'].dt.day_name()` |
| Filter by date | `df[df['date'] >= '2020-01-01']` |
| Set datetime index | `df.set_index('date')` |
| Select by date string | `ts['2020-03']` |
| Resample to monthly | `ts.resample('ME').mean()` |
| Rolling average | `ts.rolling(window=3).mean()` |
| Percent change | `ts.pct_change()` |

---

## ➡️ Next Tutorial

In **Tutorial 7: Patterns in the Darkness**, you will learn to use pivot tables and crosstabs to find patterns in the incident data—including the disturbing pattern of child disappearances near the Quarry boundary.

*"Three children in two months. All near the boundary. All between dusk and dawn."*

*The archivist set down her pen. "You think it's the creatures?"*

*"I think," said Gull, "that whatever took them moves in threes."*

---

## 💾 Save Your Work

**File > Save a copy in Drive** to keep your completed notebook.